In [1]:
%load_ext autoreload
%autoreload 2

import asyncio, copy, os, socket, sys, time
from functools import partial
from multiprocessing import Pool, Process
from pathlib import Path
from tqdm import tqdm

import torch
from torch import optim
from torch.utils.tensorboard import SummaryWriter

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
from libs import agg, data, fl, log, nn, poison, resnet, sim, wandb
from cfgs.fedargs import *

In [2]:
project = 'fl-poison'
name = 'fedavg-cnn-mnist-na'

#Define Custom CFFs

# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("info")
wb = wandb.init(name, project)

wandb: Currently logged in as: kasyah (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
# Device settings
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [4]:
# Prepare clients
host = socket.gethostname()
clients = [host + "(" + str(client + 1) + ")" for client in range(fedargs.num_clients)]

In [5]:
# Initialize Global and Client models
global_model = copy.deepcopy(fedargs.model)
# Load Data to clients
train_data, test_data = data.load_dataset(fedargs.dataset)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /home/harsh_1921cs01/hub/AgroFed/fl/libs/../data/MNIST/raw/train-images-idx3-ubyte.gz to /home/harsh_1921cs01/hub/AgroFed/fl/libs/../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /home/harsh_1921cs01/hub/AgroFed/fl/libs/../data/MNIST/raw/train-labels-idx1-ubyte.gz to /home/harsh_1921cs01/hub/AgroFed/fl/libs/../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /home/harsh_1921cs01/hub/AgroFed/fl/libs/../data/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/harsh_1921cs01/hub/AgroFed/fl/libs/../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /home/harsh_1921cs01/hub/AgroFed/fl/libs/../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/harsh_1921cs01/hub/AgroFed/fl/libs/../data/MNIST/raw

Processing...


/home/harsh_1921cs01/anaconda3/envs/syft/lib/python3.9/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


<h2>FLTrust</h2>

In [6]:
if FLTrust["is"]:
    train_data, FLTrust["data"] = data.random_split(train_data, FLTrust["ratio"])
    FLTrust["loader"] = torch.utils.data.DataLoader(FLTrust["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)
    
    if FLTrust["proxy"]["is"]:
        FLTrust["data"], FLTrust["proxy"]["data"] = data.random_split(FLTrust["data"], FLTrust["proxy"]["ratio"])
        FLTrust["loader"] = torch.utils.data.DataLoader(FLTrust["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)
        FLTrust["proxy"]["loader"] = torch.utils.data.DataLoader(FLTrust["proxy"]["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)

<h2>Prepare a backdoored loader for test</h2>

In [7]:
if backdoor_attack["is"]:
    train_data, backdoor_attack["data"] = data.random_split(train_data, backdoor_attack["ratio"])
    backdoor_attack["data"] = poison.insert_trojan(backdoor_attack["data"],
                                                   backdoor_attack["target_label"],
                                                   backdoor_attack["trojan_func"], 1)
    backdoor_attack["loader"] = torch.utils.data.DataLoader(backdoor_attack["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)

<h2>Load client's data</h2>

In [8]:
clients_data = data.split_data(train_data, clients)

<h2>Label Flip Attack</h2>

In [9]:
if label_flip_attack["is"]:
    for client in mal_clients:
        clients_data[clients[client]] = label_flip_attack["func"](clients_data[clients[client]],
                                                                  label_flip_attack["labels"],
                                                                  label_flip_attack["percent"])

<h2>Backdoor Attack</h2>

In [10]:
if backdoor_attack["is"]:
    for client in mal_clients:
        clients_data[clients[client]] = poison.insert_trojan(clients_data[clients[client]],
                                                             backdoor_attack["target_label"],
                                                             backdoor_attack["trojan_func"], 0.5)

In [11]:
client_train_loaders, _ = data.load_client_data(clients_data, fedargs.client_batch_size, None, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=fedargs.test_batch_size, shuffle=True, **kwargs)

client_details = {
        client: {"train_loader": client_train_loaders[client],
                 "model": copy.deepcopy(global_model),
                 "model_update": None}
        for client in clients
    }

In [12]:
def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def process(client, epoch, model, train_loader, fedargs, device):
    # Train
    model_update, model, loss = fedargs.train_func(model, train_loader, 
                                                   fedargs.learning_rate,
                                                   fedargs.weight_decay,
                                                   fedargs.local_rounds, device)

    log.jsondebug(loss, "Epoch {} of {} : Federated Training loss, Client {}".format(epoch, fedargs.epochs, client))
    log.modeldebug(model_update, "Epoch {} of {} : Client {} Update".format(epoch, fedargs.epochs, client))
    
    return model_update

In [13]:
import time
start_time = time.time()
    
# Federated Training
for epoch in tqdm(range(fedargs.epochs)):
    log.info("Federated Training Epoch {} of {}".format(epoch, fedargs.epochs))

    # Global Model Update
    if epoch > 0:
        # For Tmean and FLTrust, not impacts others as of now
        avgargs = {"beta": len(mal_clients), 
                   "base_model_update": global_model_update if FLTrust["is"] else None,
                   "base_norm": True}
        
        # Average
        global_model = fl.federated_avg(client_model_updates, global_model, fedargs.agg_rule, **avgargs)
        log.modeldebug(global_model, "Epoch {}: Server Update".format(epoch))
        
        # Test, Plot and Log
        global_test_output = fedargs.eval_func(global_model, test_loader, device, label_flip_attack["labels"])
        wb.log({"epoch": epoch, "time": time.time(), "acc": global_test_output["accuracy"], "loss": global_test_output["test_loss"]})
        log.jsoninfo(global_test_output, "Global Test Outut after Epoch {} of {}".format(epoch, fedargs.epochs))
        
        # Evaluate LFA
        if "attack" in global_test_output:
            if "attack_success_rate" in global_test_output["attack"]:
                wb.log({"attack_success_rate": global_test_output["attack"]["attack_success_rate"]})
            if "misclassification_rate" in global_test_output["attack"]:
                wb.log({"misclassification_rate": global_test_output["attack"]["misclassification_rate"]})

        # Evaluate Backdoor
        if backdoor_attack["is"]:
            backdoor_test_output = fl.backdoor_test(global_model, backdoor_attack["loader"], device, backdoor_attack["target_label"])
            wb.log({"backdoor_success_rate": backdoor_test_output["accuracy"]})
            log.jsoninfo(backdoor_test_output, "Backdoor Test Outut after Epoch {} of {}".format(epoch, fedargs.epochs))

        # Update client models
        for client in clients:
            client_details[client]['model'] = copy.deepcopy(global_model)

    # Clients
    tasks = [process(client, epoch, client_details[client]['model'],
                     client_details[client]['train_loader'],
                     fedargs, device) for client in clients]
    try:
        updates = fedargs.loop.run_until_complete(asyncio.gather(*tasks))
    except KeyboardInterrupt as e:
        log.error("Caught keyboard interrupt. Canceling tasks...")
        tasks.cancel()
        fedargs.loop.run_forever()
        tasks.exception()

    for client, update in zip(clients, updates):
        client_details[client]['model_update'] = update
    client_model_updates = {client: details["model_update"] for client, details in client_details.items()}
    
    # Fang attack
    if fang_attack["is"]:
        client_model_updates = fang_attack["func"](client_model_updates, len(mal_clients), fang_attack["kn"])
        
    # LIE attack
    if lie_attack["is"]:
        client_model_updates = lie_attack["func"](client_model_updates, len(mal_clients), lie_attack["kn"])
   
    # SOTA attack
    if sota_attack["is"]:
        client_model_updates = sota_attack["func"](client_model_updates, len(mal_clients), 
                                                   sota_attack["kn"], sota_attack["dev_type"])
    
    # FLtrust or FLTC based aggregation rules or attacks
    if FLTrust["is"]:
        global_model_update, _, _ = fedargs.train_func(global_model, FLTrust["loader"],
                                                       fedargs.learning_rate,
                                                       fedargs.weight_decay,
                                                       fedargs.local_rounds, device)

        # For Attacks related to FLTrust
        base_model_update = global_model_update
        if FLTrust["proxy"]["is"]:
            base_model_update, _, _ = fedargs.train_func(global_model, FLTrust["proxy"]["loader"],
                                                         fedargs.learning_rate,
                                                         fedargs.weight_decay,
                                                         fedargs.local_rounds, device)
        
        # Layer replacement attack
        if layer_replacement_attack["is"]:
            for client in mal_clients:
                client_details[clients[client]]['model_update'] = layer_replacement_attack["func"](base_model_update,
                                                                                                   client_details[clients[client]]['model_update'],
                                                                                                   layer_replacement_attack["layers"])

        # For cosine attack, Malicious Clients
        if cosine_attack["is"]:
            p_models, params_changed = cosine_attack["func"](base_model_update, cosine_attack["args"], epoch,
                                                             client_model_updates, len(mal_clients), cosine_attack["kn"])
            
            for client, p_model in enumerate(p_models):
                client_details[clients[client]]['model_update'] = p_model 

            #plot params changed for only one client
            fedargs.tb.add_scalar("Params Changed for Cosine Attack/", params_changed, epoch)

        # For sybil attack, Malicious Clients
        if sybil_attack["is"]:
            for client in mal_clients:
                client_details[clients[client]]['model_update'] = base_model_update
                
        # again pair, as changed during attack
        client_model_updates = {client: details["model_update"] for client, details in client_details.items()}

print(time.time() - start_time)

  2%|▏         | 1/51 [00:48<40:46, 48.92s/it]2022-02-04 11:04:35,010 - <ipython-input-13-00521f11c043>::<module>(l:6) : Federated Training Epoch 1 of 51 [MainProcess : MainThread (INFO)]
2022-02-04 11:04:38,941 - <ipython-input-13-00521f11c043>::<module>(l:22) : Global Test Outut after Epoch 1 of 51 {
    "accuracy": 69.76,
    "correct": 6976,
    "test_loss": 2.04204642829895
} [MainProcess : MainThread (INFO)]
  4%|▍         | 2/51 [01:39<40:48, 49.97s/it]2022-02-04 11:05:25,707 - <ipython-input-13-00521f11c043>::<module>(l:6) : Federated Training Epoch 2 of 51 [MainProcess : MainThread (INFO)]
2022-02-04 11:05:29,556 - <ipython-input-13-00521f11c043>::<module>(l:22) : Global Test Outut after Epoch 2 of 51 {
    "accuracy": 77.27000000000001,
    "correct": 7727,
    "test_loss": 1.4961984554290773
} [MainProcess : MainThread (INFO)]
  6%|▌         | 3/51 [02:31<40:33, 50.69s/it]2022-02-04 11:06:17,258 - <ipython-input-13-00521f11c043>::<module>(l:6) : Federated Training Epoch 3 of

 39%|███▉      | 20/51 [16:32<25:32, 49.44s/it]2022-02-04 11:20:18,411 - <ipython-input-13-00521f11c043>::<module>(l:6) : Federated Training Epoch 20 of 51 [MainProcess : MainThread (INFO)]
2022-02-04 11:20:21,538 - <ipython-input-13-00521f11c043>::<module>(l:22) : Global Test Outut after Epoch 20 of 51 {
    "accuracy": 92.67999999999999,
    "correct": 9268,
    "test_loss": 0.2625988066673279
} [MainProcess : MainThread (INFO)]
 41%|████      | 21/51 [17:20<24:35, 49.19s/it]2022-02-04 11:21:07,038 - <ipython-input-13-00521f11c043>::<module>(l:6) : Federated Training Epoch 21 of 51 [MainProcess : MainThread (INFO)]
2022-02-04 11:21:10,330 - <ipython-input-13-00521f11c043>::<module>(l:22) : Global Test Outut after Epoch 21 of 51 {
    "accuracy": 92.82000000000001,
    "correct": 9282,
    "test_loss": 0.25697710835933685
} [MainProcess : MainThread (INFO)]
 43%|████▎     | 22/51 [18:09<23:38, 48.92s/it]2022-02-04 11:21:55,319 - <ipython-input-13-00521f11c043>::<module>(l:6) : Federat

2022-02-04 11:35:41,103 - <ipython-input-13-00521f11c043>::<module>(l:22) : Global Test Outut after Epoch 39 of 51 {
    "accuracy": 94.99,
    "correct": 9499,
    "test_loss": 0.17889632086753846
} [MainProcess : MainThread (INFO)]
 78%|███████▊  | 40/51 [32:39<08:48, 48.02s/it]2022-02-04 11:36:26,042 - <ipython-input-13-00521f11c043>::<module>(l:6) : Federated Training Epoch 40 of 51 [MainProcess : MainThread (INFO)]
2022-02-04 11:36:29,320 - <ipython-input-13-00521f11c043>::<module>(l:22) : Global Test Outut after Epoch 40 of 51 {
    "accuracy": 95.04,
    "correct": 9504,
    "test_loss": 0.17595592032670976
} [MainProcess : MainThread (INFO)]
 80%|████████  | 41/51 [33:28<08:01, 48.13s/it]2022-02-04 11:37:14,437 - <ipython-input-13-00521f11c043>::<module>(l:6) : Federated Training Epoch 41 of 51 [MainProcess : MainThread (INFO)]
2022-02-04 11:37:17,340 - <ipython-input-13-00521f11c043>::<module>(l:22) : Global Test Outut after Epoch 41 of 51 {
    "accuracy": 95.16,
    "correct

2487.9885885715485


<h1> End </h1>